## Using CNN

# Creating Training Dataset

In [ ]:
import os
import random
from PIL import Image

# Defining the shapes and their corresponding base images
shapes = {
    'gear': 'images/gear.tif',
    'hexagon': 'images/hexagon.tif',
    'disc': 'images/disc.tif',
    'square': 'images/square.tif'
}

# Create the images
for shape, image_path in shapes.items():
    # Loading the base image
    base_image = Image.open(image_path).convert('I')  

    for i in range(1, 899):  # Create 100 images for each shape
        # Rotating and scaling the image
        angle = random.randint(0, 360)
        scale = random.uniform(0.75, 1)
        img = base_image.rotate(angle).resize((int(base_image.width * scale), int(base_image.height * scale)))

        # Saving the image
        directory = f'training_set/{shape}'
        if not os.path.exists(directory):
            os.makedirs(directory)
        img.save(f'{directory}/{shape}{i}.png')


# Training the Model

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=4, activation='softmax'))  # 4 units for 4 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Using ImageDataGenerator to preprocess the images
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Directory 'training_set' with subdirectories 'gear', 'hexagon', 'disc', 'square' each containing relevant images
training_set = train_datagen.flow_from_directory('training_set', target_size=(64, 64), batch_size=32, class_mode='categorical')

# Fit the model
model.fit(training_set, steps_per_epoch=8000, epochs=25)




2024-03-03 20:49:12.691493: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-03 20:49:12.739659: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-03 20:49:12.740834: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 20:49:13.748578: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 403 images belonging to 4 classes.
Epoch 1/25
8000/8000 [==============================] - 2s 141us/step - loss: 2.3363 - accuracy: 0.3573


# Prediction

In [12]:
from keras.preprocessing import image
import numpy as np

# Load the image
img_path = 'gear1.png'  # Replace with your image path
img = image.load_img(img_path, target_size=(64, 64))

# Preprocess the image
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.  # Remember that the model was trained on inputs that were rescaled by 1./255

#Predict the class of the image
prediction = model.predict(img_tensor)

# The prediction will be a 4-element vector representing the probability of each class.
# To get the class with the highest probability, you can use np.argmax
predicted_class = np.argmax(prediction)

print(predicted_class)
# predicted_class will be 0, 1, 2, or 3 representing 'gear', 'hexagon', 'disc', 'square' respectively.


1/1 [==============================] - 0s 21ms/step
1
